## 4.4 평가


In [ ]:
# 공통 처리

# 불필요한 경고 메시지 무시
import warnings
warnings.filterwarnings('ignore')

# 라이브러리 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 한글 글꼴 설정
import platform

if platform.system() == 'Windows':
    plt.rc('font', family='Malgun Gothic')
elif platform.system() == 'Darwin':
    plt.rc('font', family='Apple Gothic')

# 데이터프레임 출력용 함수
from IPython.display import display

# 숫자 출력 조정
# 넘파이 부동소수점 출력 자리수 설정
np.set_printoptions(suppress=True, precision=4)

# 판다스 부동소수점 출력 자리수 설정
pd.options.display.float_format = '{:.4f}'.format

# 데이터프레임 모든 필드 출력
pd.set_option("display.max_columns",None)

# 그래프 글꼴 크기 설정
plt.rcParams["font.size"] = 14

# 난수 시드
random_seed = 123

### 4.4.1 혼동행렬

#### 혼동행렬

In [ ]:
# 데이터 읽어 들이기 및 데이터 분할

# 라이브러리를 임포트
from sklearn.datasets import load_breast_cancer

# 데이터 내려받기
cancer = load_breast_cancer()

# 입력 데이터 x
x = cancer.data

# 정답 데이터 y
# 양성: 0 악성: 1이 되도록 값을 변경
y = 1 - cancer.target

# 입력 데이터를 2차원으로 추려냄
x2 = x[:,:2]

# (4) 데이터 분할
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x2, y,
    train_size=0.7, test_size=0.3, random_state=random_seed)

In [ ]:
# 알고리즘 선택 및 평가

# 알고리즘 선택(로지스틱 회귀)
from sklearn.linear_model import LogisticRegression
algorithm = LogisticRegression(random_state=random_seed)

# 학습
algorithm.fit(x_train, y_train)

# 예측
y_pred = algorithm.predict(x_test)

# 평가
score = algorithm.score(x_test, y_test)

# 결과 확인
print(f'score: {score:.4f}')

In [ ]:
# 혼동행렬 계산하기

# 필요한 라이브러리를 추가 임포트
from sklearn.metrics import confusion_matrix

# 혼동행렬 작성하기
#   y_test: 검증 데이터의 정답 데이터
#   y_pred: 검증 데이터에 대한 예측 결과
matrix = confusion_matrix(y_test, y_pred)

# 결과 확인
print(matrix)

In [ ]:
# 혼동행렬을 깔끔하게 출력하는 유틸리티 함수

def make_cm(matrix, columns):
    # matrix 넘파이 배열

    # columns 필드명 목록
    n = len(columns)

    # '정답 데이터'가 n번 반복되는 리스트를 생성
    act = ['정답 데이터'] * n
    pred = ['예측 결과'] * n

    # 데이터프레임을 생성
    cm = pd.DataFrame(matrix,
        columns=[pred, columns], index=[act, columns])
    return cm

In [ ]:
# make_cm 함수를 사용해 혼동행렬 출력하기
cm = make_cm(matrix, ['양성', '악성'])
display(cm)

### 4.4.2 정확도, 정밀도, 재현율, F-점수

In [ ]:
# 정밀도, 재현율, F-점수 계산하기

# 라이브러리 임포트
from sklearn.metrics import precision_recall_fscore_support

# 정밀도, 재현율, F-점수 계산
precision, recall, fscore, _ = precision_recall_fscore_support(
    y_test, y_pred, average='binary')

# 계산 결과 확인
print(f'정밀도: {precision:.4f}')
print(f'재현율: {recall:.4f}')
print(f'F-점수: {fscore:.4f}')

### 4.4.3 확률값과 역치

In [ ]:
# 모델 내부의 확률값 구하기
y_proba = algorithm.predict_proba(x_test)
print(y_proba[10:20,:])

In [ ]:
# 양성(y=1)일 확률값을 꺼냄
y_proba1 = y_proba[:,1]

# 결과 확인
print(y_pred[10:20])
print(y_proba1[10:20])

In [ ]:
# 역치를 변경한다
thres = 0.5
print((y_proba1[10:20] > thres).astype(int))

thres = 0.7
print((y_proba1[10:20] > thres).astype(int))

In [ ]:
# 변경된 역치를 적용하는 예측 함수를 정의함
def pred(algorithm, x, thres):

    # 확률값 꺼내기(행렬)
    y_proba = algorithm.predict_proba(x)

    # 예측 결과 1에 대한 확률값
    y_proba1 =  y_proba[:,1]

    # 예측 결과 1에 대한 확률값 > 역치
    y_pred = (y_proba1 > thres).astype(int)
    return y_pred

In [ ]:
# 역치 0.5를 적용한 예측 결과
pred_05 = pred(algorithm, x_test, 0.5)

# 역치 0.7을 적용한 예측 결과
pred_07 = pred(algorithm, x_test, 0.7)

# 결과 확인
print(pred_05[10:20])
print(pred_07[10:20])

### 4.4.4 PR 곡선과 ROC 곡선

#### PR 곡선

In [ ]:
# PR 곡선을 그리기 위한 배열을 생성

# 라이브러리 임포트 
from sklearn.metrics import precision_recall_curve
# 정밀도, 재현율, 역치를 계산
precision, recall, thresholds = precision_recall_curve(
    y_test, y_proba1)
# 계산 결과를 데이터프레임으로 변환
df_pr = pd.DataFrame([thresholds, precision, recall]).T
df_pr.columns = ['역치', '정밀도', '재현율']
# 역치 0.5 주변의 값을 출력
display(df_pr[52:122:10])

In [ ]:
# PR 곡선 그리기

# 그래프 크기를 설정
plt.figure(figsize=(6,6))
# 그래프 영역을 색으로 채우기
plt.fill_between(recall, precision, 0)
# x, t의 범위 지정
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
# 레이블, 제목 추가
plt.xlabel('재현율')
plt.ylabel('정밀도')
plt.title('PR 곡선')
plt.show()


In [ ]:
# PR 곡선 아래 영역의 면적 계산
from sklearn.metrics import auc
pr_auc = auc(recall, precision)
print(f'PR 곡선 아래 영역의 면적: {pr_auc:.4f}')

#### ROC 곡선

In [ ]:
# ROC 곡선을 그리기 위한 배열을 생성

# 라이브러리 임포트 
from sklearn.metrics import roc_curve

# 위양성률, 민감도, 역치를 계산
fpr, tpr, thresholds = roc_curve(
    y_test, y_proba1,drop_intermediate=False)

# 계산 결과를 데이터프레임으로 변환
df_roc = pd.DataFrame([thresholds, fpr, tpr]).T
df_roc.columns = ['역치', '위양성률', '민감도']

# 역치 0.5 주변의 값을 출력
display(df_roc[21:91:10])

In [ ]:
# ROC 곡선 그리기

# 그래프 크기를 설정
plt.figure(figsize=(6,6))

# 점선 그리기
plt.plot([0, 1], [0, 1], 'k--')

# 그래프 영역을 색으로 채우기
plt.fill_between(fpr, tpr, 0)

# x, y의 범위 지정
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])

# 레이블, 제목 추가
plt.xlabel('위양성률')
plt.ylabel('민감도')
plt.title('ROC 곡선')
plt.show()

In [ ]:
# ROC 곡선 아래의 면적을 계산
roc_auc = auc(fpr, tpr)
print(f'ROC 곡선 아래의 면적:{roc_auc:.4f}')

#### 성능이 뛰어난 모델의 ROC 곡선 그리기

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
  x, y, train_size=0.7, test_size=0.3, random_state=random_seed)
algorithm = LogisticRegression()
algorithm.fit(x_train, y_train)
y_pred = algorithm.predict(x_test)
y_proba1 = algorithm.predict_proba(x_test)[:,1]
fpr, tpr, thresholds = roc_curve(y_test, y_proba1)

In [ ]:
# ROC 곡선 그리기

plt.figure(figsize=(6,6))
plt.plot([0, 1], [0, 1], 'k--')
plt.fill_between(fpr, tpr, 0)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('위양성률')
plt.ylabel('민감도')
plt.title('ROC곡선')
plt.show()

In [ ]:
# ROC 곡선 아래의 면적을 계산
roc_auc = auc(fpr, tpr)
print(f'ROC 곡선 아래의 면적:{roc_auc:.4f}')

### 4.4.5 입력 필드의 중요도

#### 랜덤 포레스트 모델 생성 단계까지

In [ ]:
# 랜덤 포레스트 모델 생성까지

# 예제 데이터 읽어 들이기
import seaborn as sns
df_iris = sns.load_dataset("iris")
columns_i = ['꽃받침_길이', '꽃받침_폭', '꽃잎_길이', '꽃잎_폭', '종']
df_iris.columns = columns_i

# 입력 데이터 x
x = df_iris[['꽃받침_길이', '꽃받침_폭', '꽃잎_길이', '꽃잎_폭']]

# 정답 데이터 y
y = df_iris['종']

# 알고리즘 선택 (랜덤 포레스트)
from sklearn.ensemble import RandomForestClassifier
algorithm = RandomForestClassifier(random_state=random_seed)

# 학습
algorithm.fit(x, y)

#### 중요도 벡터

In [ ]:
# 중요도 벡터 계산하기
importances = algorithm.feature_importances_

# 필드명이 키인 Series 객체를 생성
w = pd.Series(importances, index=x.columns)

# 내림차순으로 정렬
u = w.sort_values(ascending=False)

# 결과 확인
print(u)

In [ ]:
# 막대그래프로 중요도 나타내기

# 막대그래프 그리기
plt.bar(range(len(u)), u, color='b', align='center')

# 필드명 출력(90도 회전) 
plt.xticks(range(len(u)), u.index, rotation=90)

# 제목 출력
plt.title('입력변수의 중요도')

plt.show()

In [ ]:
# 결정 트리 모델

from sklearn.tree import DecisionTreeClassifier
algorithm = DecisionTreeClassifier(random_state=random_seed)
algorithm.fit(x, y)
importances = algorithm.feature_importances_
w = pd.Series(importances, index=x.columns)
u = w.sort_values(ascending=False)
plt.title('입력변수의 중요도(결정 트리)')
plt.bar(range(len(u)), u, color='b', align='center')
plt.xticks(range(len(u)), u.index, rotation=90)
plt.show()

In [ ]:
# XGBoost

import xgboost
algorithm = xgboost.XGBClassifier(random_state=random_seed)
algorithm.fit(x, y)
importances = algorithm.feature_importances_
w = pd.Series(importances, index=x.columns)
u = w.sort_values(ascending=False)
plt.title('입력변수의 중요도(XGBoost)')
plt.bar(range(len(u)), u, color='b', align='center')
plt.xticks(range(len(u)), u.index, rotation=90)
plt.show()

### 4.4.6 회귀 모델을 평가하는 방법

In [ ]:
# 데이터 읽어 들이기부터 데이터 분할 단계까지

# 데이터 읽어 들이기 (보스턴 데이터 집합)
from sklearn.datasets import load_boston
boston = load_boston()

# df: 입력 데이터
df = pd.DataFrame(boston.data, columns=boston.feature_names)

# y: 정답 데이터
y = boston.target

# 입력 필드가 1개 뿐인 입력 데이터 df1
df1 = df[['RM']]

# 결과 확인
display(df.head())
display(df1.head())
print(y[:5])

In [ ]:
# 알고리즘 선택부터 예측까지

# 알고리즘: XGBRegressor
from xgboost import XGBRegressor
algorithm1 = XGBRegressor(objective ='reg:squarederror',
    random_state=random_seed)

# 학습 (df1을 입력)
algorithm1.fit(df1, y)

# 예측
y_pred1 = algorithm1.predict(df1)


# 알고리즘: XGBRegressor
from xgboost import XGBRegressor
algorithm2 = XGBRegressor(objective ='reg:squarederror',
    random_state=random_seed)

# 학습 (df를 입력)
algorithm2.fit(df, y)

# 예측
y_pred2 = algorithm2.predict(df)

In [ ]:
# 결과 확인
print(f'y[:5] {y[:5]}')
print(f'y_pred1[:5] {y_pred1[:5]}')
print(f'y_pred2[:5] {y_pred2[:5]}')

In [ ]:
# y의 최댓값과 최솟값을 계산
y_range = np.array([y.min(), y.max()])

# 결과 확인
print(y_range)

#### 산점도 그리기

#### 입력변수가 1개인 경우

In [ ]:
# 산점도를 이용한 결과 확인(입력변수 1개)

# 그래프 사이즈 설정
plt.figure(figsize=(6,6))

# 산점도 그리기
plt.scatter(y, y_pred1)

# 정답 데이터=예측결과 보조선 그리기
plt.plot(y_range, y_range, 'k--')

# 레이블, 제목 표시
plt.xlabel('정답 데이터')
plt.ylabel('예측 결과')
plt.title('정답 데이터와 예측 결과의 산점도(입력변수 1개)')

plt.show()

#### 입력변수가 13개인 경우

In [ ]:
# 산점도를 이용한 결과 확인(입력변수 13개)

# 그래프 사이즈 설정
plt.figure(figsize=(6,6))

# 산점도 그리기
plt.scatter(y, y_pred2)

# 정답 데이터=예측결과 보조선 그리기
plt.plot(y_range, y_range, 'k--')

# 레이블, 제목 표시
plt.xlabel('정답 데이터')
plt.ylabel('예측 결과')
plt.title('정답 데이터와 예측 결과의 산점도(입력변수 13개)')

plt.show()

#### R2 score

In [ ]:
# r2 값 계산 (입력변수 1개)
from sklearn.metrics import r2_score
r2_score1 = r2_score(y, y_pred1)
print(f'R2 값(입력변수 1개): {r2_score1:.4f}')

In [ ]:
# r2 값 계산 (입력변수 13게)
r2_score2 = r2_score(y, y_pred2)
print(f'R2 값(입력변수 13개): {r2_score2:.4f}')